In [201]:
import sys
sys.path.append('../')

from utils.GetData import Data
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [172]:
PATH = '../in/train.csv'
df = pd.read_csv(PATH, index_col='Id')

In [51]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [52]:
data = Data()

In [68]:
df_ = df.drop(data.bad_features, axis=1)
object_cols = df.select_dtypes('object').columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [259]:

LotShape = ['IR3','IR2','IR1','Reg']
Utilities = ['ELO','NoSeWa','NoSewr','AllPub']
LandSlope = ['Sev','Mod','Gtl']
HouseStyle = ['1Story','1.5Fin','1.5Unf','2Story','2.5Fin','2.5Unf','SFoyer','SLvl'] # posible oh
ExterQual = ['Po','Fa','TA','Gd','Ex']
ExterCond = ['Po','Fa','TA','Gd','Ex']
BsmtQual = ['NA','Po','Fa','TA','Gd','Ex'] 
BsmtCond = ['NA','Po','Fa','TA','Gd','Ex'] 
BsmtExposure = ['NA','No','Mn','Av','Gd']
BsmtFinType1 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'] 
BsmtFinType2 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'] 
HeatingQC = ['Po','Fa','TA','Gd','Ex']
CentralAir = ['N','Y']
Electrical = ['Mix','FuseP','FuseF','FuseA','SBrkr']  
KitchenQual = ['Po','Fa','TA','Gd','Ex']
Functional = ['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ']
FireplaceQu = ['NA','Po','Fa','TA','Gd','Ex'] 
GarageType = ['NA','Detchd','CarPort','BuiltIn','Basment','Attchd','2Types'] 
GarageFinish = ['NA','Unf','RFn','Fin'] 
GarageQual = ['NA','Po','Fa','TA','Gd','Ex'] 
GarageCond = ['NA','Po','Fa','TA','Gd','Ex']
PavedDrive = ['N','P','Y']
PoolQC = ['NA','Fa','TA','Gd','Ex'] 
Fence = ['NA','MnWw','GdWo','MnPrv','GdPrv']

# fill Electrical na values with 'most frecuente'
# fil GarageYrBlt na values with most frecuente
# change value None of MasVnrType to NA

col_fill_NA = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
col_fill_0 = ['LotFrontage','MasVnrArea','BsmtFullBath','BsmtHalfBath','BsmtFinSF2','GarageArea','BsmtFinSF1','GarageCars','TotalBsmtSF','BsmtUnfSF']
col_fill_mf = ['MSZoning','Electrical','GarageYrBlt','Functional','Utilities','Exterior2nd','Exterior1st','SaleType','KitchenQual']

ordinal_cols_categories = [LotShape,Utilities,LandSlope, HouseStyle,ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence]
ordinal_cols = ['LotShape','Utilities','LandSlope', 'HouseStyle','ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']

# antes de ejecutar oh, rellenar los na de MiscFeature y Alley con 'NA'
oh_cols = ['MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood', 'Condition1','Condition2','BldgType','RoofStyle','RoofMatl','Exterior1st','Exterior2nd', 'MasVnrType','Foundation','Heating','MiscFeature','SaleType','SaleCondition']

In [55]:
len(col_fill_NA) + len(col_fill_0)

17

In [94]:
df[col_fill_NA] = df[col_fill_NA].fillna('NA')
df[col_fill_0] = df[col_fill_0].fillna(0)

df.replace('None', 'NA')

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,MnPrv,NA,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [260]:
df = pd.read_csv(PATH)

In [249]:
df.isna().sum().sort_values(ascending=False)[:20]

Id              0
CentralAir      0
GarageYrBlt     0
GarageType      0
FireplaceQu     0
Fireplaces      0
Functional      0
TotRmsAbvGrd    0
KitchenQual     0
KitchenAbvGr    0
BedroomAbvGr    0
HalfBath        0
FullBath        0
BsmtHalfBath    0
BsmtFullBath    0
GrLivArea       0
LowQualFinSF    0
2ndFlrSF        0
1stFlrSF        0
GarageFinish    0
dtype: int64

In [261]:
imputer_N = SimpleImputer(strategy='constant',fill_value='NA')
df[col_fill_NA] = pd.DataFrame(imputer_N.fit_transform(df[col_fill_NA]), columns=col_fill_NA)

imputer_0 = SimpleImputer(strategy='constant',fill_value=0)
df[col_fill_0] = pd.DataFrame(imputer_0.fit_transform(df[col_fill_0]), columns=col_fill_0)

imputer_mf = SimpleImputer(strategy='most_frequent')
df[col_fill_mf] = pd.DataFrame(imputer_mf.fit_transform(df[col_fill_mf]), columns=col_fill_mf)


df['MasVnrType'].replace('None', 'NA', inplace=True)


In [262]:
ordinal = OrdinalEncoder(categories= ordinal_cols_categories)
df[ordinal_cols] = pd.DataFrame(ordinal.fit_transform(df[ordinal_cols]), columns=ordinal_cols)

In [263]:
oh_encoder = OneHotEncoder(handle_unknown='ignore')
df_oh = pd.DataFrame(oh_encoder.fit_transform(df[oh_cols]).toarray())

In [252]:
df.shape

(1460, 81)

In [264]:
df.drop(oh_cols, axis=1, inplace=True)
X = pd.concat([df,df_oh], join='inner', axis=1)

In [265]:
X.shape

(1460, 209)

In [194]:
X.head()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,Utilities,LandSlope,HouseStyle,OverallQual,OverallCond,...,137,138,139,140,141,142,143,144,145,146
0,1,60,65.0,8450,3.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,80.0,9600,3.0,3.0,2.0,0.0,6,8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,68.0,11250,2.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,60.0,9550,2.0,3.0,2.0,3.0,7,5,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,84.0,14260,2.0,3.0,2.0,3.0,8,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [195]:

X = X.drop(columns=['SalePrice','Id'], axis=1)

discrete_features = X.dtypes == int
y = df['SalePrice']

In [196]:
X.shape

(1460, 207)

In [197]:
mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)

In [198]:
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [157]:
import numpy as np

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks, fontsize=10)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 15))
plot_mi_scores(mi_scores)

In [202]:
model = RandomForestRegressor()

parameters = {
    'n_estimators': range(13,15),
    'criterion': ['squared_error','absolute_error'],
    'max_depth': range(8,11)
}

rank_estimator = RandomizedSearchCV(model, parameters, n_iter=10, cv=5, scoring='neg_mean_absolute_error')
rank_estimator.fit(X, y)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': range(8, 11),
                                        'n_estimators': range(13, 15)},
                   scoring='neg_mean_absolute_error')

In [204]:
rank_estimator.best_params_

{'n_estimators': 14, 'max_depth': 10, 'criterion': 'squared_error'}

In [266]:
pred_example = pd.read_csv('../in/sample_submission.csv')

In [267]:
test = pd.read_csv('../in/test.csv')

In [268]:
test[col_fill_NA] = pd.DataFrame(imputer_N.transform(test[col_fill_NA]), columns=col_fill_NA)

test[col_fill_0] = pd.DataFrame(imputer_0.transform(test[col_fill_0]), columns=col_fill_0)

test[col_fill_mf] = pd.DataFrame(imputer_mf.transform(test[col_fill_mf]), columns=col_fill_mf)


test['MasVnrType'].replace('None', 'NA', inplace=True)


In [271]:
test[ordinal_cols] = pd.DataFrame(ordinal.transform(test[ordinal_cols]), columns=ordinal_cols)

In [272]:
test_oh = pd.DataFrame(oh_encoder.transform(test[oh_cols]).toarray())

In [273]:
test.drop(oh_cols, axis=1, inplace=True)

In [274]:
X_test = pd.concat([test,test_oh], join='inner', axis=1)

In [276]:
X_test.drop('Id', axis=1, inplace=True)

In [277]:
predict = pd.DataFrame(rank_estimator.predict(X_test), columns=['SalePrice'])
predict['Id'] = index=pred_example.Id
predict = predict.set_index('Id')
predict.to_csv('../out/one-hot.csv')
predict.head()

,SalePrice
Id,
1461,126407.372200
1462,150179.771392
1463,182735.508287
1464,177605.987734
1465,206948.985053


Condition2 llenar valores nulos con 0

LotFrontage     